# Exploring the Air Quality Dataset
Air pollution is a major concern for the modern world. 

Prolonged exposure to low quality air resulting from  a variety of sources like vehicle emissions, fumes from chemical production, mold spores, and wildfires can lead to several adverse health effects. Some of the health effects from exposure to poor air quality are irritation of the eyes, nose and throat, shortness of breath, afflictions on the cardiovascular system and even more serious complications after long periods of time. 

<img src = "pollution air.png">

Apart from the impact that low air quality has on human health, the environmental effects of poor air quality are just as severe. Poor air quality can lead to reduced crop yields as well as increased susceptibility of plants to pests and disease amongst other things.

As such keeping a close eye on air quality levels is of import to health and the environment. 

In this project I'll be exploring the AirQuality Dataset in order to train a machine learning (ML) model capable of predicting predicting the temperature based on the concentrations of a variety of pollutants like metal oxides and hydrocarbons. This will also give me chance to work on time series data. 

The data was obtained from the UCI Machine Learning Repository here <a href = "https://archive.ics.uci.edu/ml/datasets/Air+Quality">https://archive.ics.uci.edu/ml/datasets/Air+Quality</a>. This dataset covers sensor collection data on a variety of pollutants spanning from March 2004 to February 2005 (1 year) 

## Modules Used in this work

In [ ]:
#General modules
import numpy as np
import pandas as pd
from tqdm import *

#For plotting
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import seaborn as sns

#For dealing with time string types
import datetime

#For Exploratory Data Analysis (EDA)
from pandas_profiling import ProfileReport

#For building ML model
from sklearn.model_selection import train_test_split

#Different Regressors for ML model
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel , RBF
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

#For model evaluation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict

#For parameter grid search optimization
from sklearn.model_selection import GridSearchCV

#For saving model
import pickle

#For q-q plot
import scipy.stats as stats

## Loading the Air Quality Data
I'll start by loading the data. The data was in .csv format which can be readily loaded into a pandas dataframe using the <code>read_csv</code> function.

In [ ]:
aq = pd.read_csv("AirQualityUCI.csv", sep = ";", decimal = ",")        
aq.head()

Ok. The data has 9357 observations and 17 different columns. However, there are two columns (labeled as Unnamed 15 and 16) that have garbage on them that I'll have to remove. different attributes.

In [ ]:
aq.drop(['Unnamed: 15','Unnamed: 16'], axis=1, inplace=True, errors = 'ignore') 
aq.head()

Good. Those columns are gone. Next, I'll check for null values and ensure that all the data is properly formatted. 
## Removing Null/Faulty Sensor Readings
From the dataset documentation,it is known that faulty or missing sensor readings were filled with the value '-200'. Therefore, I'll handle those instances. 

In [ ]:
#Replacing bad sensor readings designated by an entry of -200 with NaN
aq.replace(to_replace = -200, value = np.nan, inplace = True)
aq.info()

Looks like Column 4 labeled as 'NMHC(GT) has a lot of faulty readings relative to all the other sensors. Let me determine what the percent composition is of nulls with respect to the total values. This will help me determine whether to keep that column or drop it.

In [ ]:
percent_NaN = []
columns = aq.columns
for col in columns:
    pNaN =  (aq[col].isna().sum()/aq.shape[0]) * 100 #sum NaN instances in each column. Divide by total rows
    percent_NaN.append(pNaN)
nan_percent_df = pd.DataFrame(percent_NaN,
                              index=columns,
                              columns=['%_NaN_in_Column']).sort_values('%_NaN_in_Column',ascending = False)
nan_percent_df

Yeah... more than 90% of the entries in that columns are null. If I were to use the dropna method on the entire dataset I'd end up losing 90% of our data. Furthermore, we still have a column with information about Benzene (C6H6) concentrations (benzene is a hydrocarbon) through the columns C6H6(GT) and PT08.S2(NMHC), so we haven't completely lost the entirety of the hydrocarbon readings. Instead, I'll opt for dropping this feature from the dataset, dropping the NaN rows and press forward. 

In [ ]:
aq.drop('NMHC(GT)', axis=1, inplace=True, errors = 'ignore') 
aq = aq.dropna()
aq.head()

Ok, let's check the dataframe info again to see how things are looking

In [ ]:
aq.info()

Hmm the data in the Date and Time columns should be formatted as datetime objects and they are not. I'll need to fix that.
## Cleaning Up Time Features

In [ ]:
aq['DateTime'] =  (aq.Date) + ' ' + (aq.Time)
aq.DateTime = aq.DateTime.apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y %H.%M.%S'))
aq

I tried several different strategies to try to convert the Time column to an actual time. The method below was the one that gave me the result I wanted. Now, I'll drop the original Date and Time Columns and generate columns for Day, Month, and Time from the DateTime column I just made.

In [ ]:
aq['Weekday'] = aq['DateTime'].dt.day_name()
aq['Month']   = aq['DateTime'].dt.month_name()
aq['Hour']    = aq['DateTime'].dt.hour
aq['Date']    = pd.to_datetime(aq['Date'], format='%d/%m/%Y')
aq.drop('Time', axis=1, inplace=True, errors = 'ignore') 
aq.head()

Almost there. Now I'll just reorder the columns a bit and then I'm good to go. 

In [ ]:
aq = aq[['Date','Month', 'Weekday','DateTime', 'Hour', 'CO(GT)','PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)', 
         'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']]
aq.head()

Neat! And let's do a final check of formatting before proceeding

In [ ]:
aq.info()

Perfect! Now all the data is nice and tidy. 

## Checkpoint for DataFrame Profiling
These are the data features that we have at our disposal from this dataset. 

* 0 Date (DD/MM/YYYY)
* 1 Time (HH.MM.SS)
* 2 True hourly averaged concentration CO in mg/m^3 (reference analyzer)
* 3 PT08.S1 (tin oxide) hourly averaged sensor response (nominally CO targeted)
* 4 True hourly averaged overall Non-metallic HydroCarbons concentration in microg/m^3 (reference analyzer)
* 5 True hourly averaged Benzene concentration in microg/m^3 (reference analyzer)
* 6 PT08.S2 (titania) hourly averaged sensor response (nominally NMHC targeted)
* 7 True hourly averaged NOx concentration in ppb (reference analyzer)
* 8 PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)
* 9 True hourly averaged NO2 concentration in microg/m^3 (reference analyzer)
* 10 PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted)
* 11 PT08.S5 (indium oxide) hourly averaged sensor response (nominally O3 targeted)
* 12 Temperature in °C
* 13 Relative Humidity (%)
* 14 AH Absolute Humidity

Next, I'll use pandas profiling to quickly get a report on the data we have. This will give me a quick overview of what variables would be interesting to explore for a model. 


In [ ]:
aq.reset_index(drop=True, inplace=True) #Drop index prior to generating report
report = ProfileReport(aq)
report

There's a few interesting things that report has shown. 

In particular, it can be seen that all the pollutant concentrations detected by the sensors are highly correlated with one another. Interestingly, for some reason the readings from the sensor 'PT08.S3(NOx)' are anticorrelated with one another. This would suggest that the levels of Nitrous Oxide(s) are decreasing with increasing levels of the other pollutants. Why would this be? Is this a real effect?

Other things that are noteworthy are the distribution shapes for each variable. Most of them appear to have approximately unimodal distributions (that also show right-skewedness). Some of the exceptions are PT08.S4(NO2) are AH where there is fairly clear bimodal behavior occurring. This suggests that there is further cleaning up that I need to do since skewedness in a distribution is related to the presence of outliers. 
## Removing Outliers From Dataframe
I'll deal with the outliers by using the interquartile range (IQR).

In [ ]:
#Removing Outliers with the Interquartile Range Method (IQR)
Q1 = aq.quantile(0.25) #first 25% of the data
Q3 = aq.quantile(0.75) #first 75% of the data
IQR = Q3 - Q1 #IQR = InterQuartile Range

scale = 1.4 #May need to play with this value to modify outlier detection sensitivity if need be
lower_lim = Q1 - scale*IQR
upper_lim = Q3 + scale*IQR

cols = aq.columns[5:] # Look for oulierts in columns starting from CO(GT)

#Mask a masking condition that removes rows that have values above/below IQR limits
condition = ~((aq[cols] < lower_lim) | (aq[cols] > upper_lim)).any(axis=1)

#Generate new dataframe that has had its outliers removed
aq_filtered = aq[condition]
aq_filtered#.head()

In [ ]:
aq_filitered.info()

Cool. Now I've removed outliers and done my due dilligence to ensure the data is clean and ready to be processed. 

Let me generate another profile on the cleaned up data to ensure that we've properly dealt with skewedness.

In [ ]:
aq_filtered.reset_index(drop=True, inplace=True) #Drop index prior to generating report
report = ProfileReport(aq_filtered)
report

Hmmm, I've played around with different outlier sensitivities using the IQR and CO(GT), NOx(GT) and C6H6 columns still have several outliers causing skewedness. 

As I mentioned earlier, C6H6 is a hydrocarbon and we have another sensor that is detecting nonmetallic hydrocarbons (NMHC). Therefore, the C6H6 column is sorta redundant and we can drop it without losing too much information. 

With NOx, we still have a sensor in the dataframe responsible for detecting nitrous oxides. This means that the NOx column is similarly redundant.

With CO(GT), we still have a sensor in the dataframe responsible for specifically detecting CO. This means that the CO(GT) column is also redundant.

Using that same logic, I'll drop the NO2(GT) column as well. 

As such, I'll remove them from the dataset.

In [ ]:
aq_filtered.drop(['CO(GT)']  ,axis=1, inplace=True, errors = 'ignore')
aq_filtered.drop(['NOx(GT)'] ,axis=1, inplace=True, errors = 'ignore')
aq_filtered.drop(['C6H6(GT)'],axis=1, inplace=True, errors = 'ignore')
aq_filtered.drop(['NO2(GT)'] ,axis=1, inplace=True, errors = 'ignore')
aq_filtered.head()

Ok, I'll generate one final report before continuing to check the state of our data.

In [ ]:
aq_filtered.reset_index(drop=True, inplace=True) #Drop index prior to generating report
report = ProfileReport(aq_filtered)
report

Nice! The variables are now nice and normal, I've dealt with outliers and there are some interesting relationships between variables that I can explore as part of the model I'll build.

Some questions that may be interesting to look into are:

* Can we predict the level of a pollutant based on the concentrations of other pollutants?
* At what times/days do pollutant levels maximize/minimize?
* Do all pollutants increase together or do some of them peak/valley at different points?
* Is there any relationship/trend we can infer/deduce with regards to the humidity/temperature from pollutant concentrations?

I think I'll start by looking into the behavior of each of these concentrations as a function of time

## Time Series on Air Quality Data
Since one of the questions I'm interested is seeing if there are certain days/months that have worse pollution than others, I'll add a 'day' column to my dataframe before anything else.

Let's start visualizing stuff! I'm going to start by checking the CO concentrations on a monthly, daily and hourly basis.

In [ ]:
month_df_list = []
day_df_list   = []
hour_df_list  = []

months = ['January','February','March', 'April', 'May','June', 
          'July', 'August', 'September', 'October', 'November', 'December']

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for month in months:
    temp_df = aq_filtered.loc[(aq_filtered['Month'] == month)]
    month_df_list.append(temp_df)

for day in days:
    temp_df = aq_filtered.loc[(aq_filtered['Weekday'] == day)]
    day_df_list.append(temp_df)

for hour in range(24):
    temp_df = aq_filtered.loc[(aq_filtered['Hour'] == hour)]
    hour_df_list.append(temp_df)

Sweet! Now I can start visualizing the data attributes at different temporal specificities. I'll make a  function that will allow me to visualize the different pollutant concentrations at different months, days, and hours

In [ ]:
def df_time_plotter(df_list, time_unit, y_col):
    
    months = ['January','February','March', 'April', 'May','June', 
              'July', 'August', 'September', 'October', 'November', 'December']
    
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    if time_unit == 'M':
        nRows = 3
        nCols = 4
        n_iter = len(months)
    elif time_unit == 'D':
        nRows = 2
        nCols = 4
        n_iter = len(days)
    elif time_unit == 'H':
        nRows = 4
        nCols = 6
        n_iter = 24
    else:
        print('time_unit must be a string equal to M,D, or H')
        return 0
        
    fig, axs = plt.subplots(nrows=nRows, ncols=nCols, figsize = (40,30))
    axs = axs.ravel()
    for i in range(n_iter):
        data = df_list[i]
        ax = axs[i]
        data.plot(kind ='scatter', x = 'DateTime', y= y_col , ax = ax, fontsize = 24)
        ax.set_ylabel('Pollutant Concentration',fontsize=30)
        ax.set_xlabel('')
        if time_unit == 'M':
            ax.set_title(y_col + ' ' + months[i],  size=40) # Title
        elif time_unit == 'D':
            ax.set_title(y_col + ' ' + days[i],  size=40) # Title
        else:
             ax.set_title(y_col + ' ' + str(i),  size=40) # Title
        ax.tick_params(labelrotation=60)

        #plt.xlim([datetime.date(2004, 3, 10), datetime.date(2004, 3, 30)])
    # set the spacing between subplots
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.5)
    plt.show() # Depending on whether you use IPython or interactive mode, etc.

Let's see how the the CO levels change every month

In [ ]:
df_time_plotter(month_df_list,'M','PT08.S1(CO)')

Hmmm, the CO levels are highest in November and December. The sensors had a lot of missing/bad readings on March and April. Let's see how the concentrations change for every day of the week,|

In [ ]:
df_time_plotter(day_df_list,'D','PT08.S1(CO)')

Hmm, the CO levels are lowest on Saturday and Sunday it appears. Let's see it as a function of time.

In [ ]:
df_time_plotter(hour_df_list,'H','PT08.S1(CO)')

It looks like readings are quite low between 4-6 AM. The CO levels rise starting at 1PM and peak at around 6-8pm. Maybe a bar plot will make some of these relationships more apparent

In [ ]:
sns.barplot(x = 'Month', y = 'PT08.S1(CO)', data = aq_filtered)
plt.title('CO Values Per Month')
plt.xticks(rotation=90)
plt.show()

sns.barplot(x = 'Weekday', y = 'PT08.S1(CO)', data = aq_filtered)
plt.title('CO Values Per Day of the Week')
plt.xticks(rotation=90)
plt.show()

sns.barplot(x = 'Hour', y = 'PT08.S1(CO)', data = aq_filtered)
plt.title('CO Values Per Hour')
plt.xticks(rotation=90)
plt.show()

This is a lot clearer. 

From these plots we can see the following:

* October has the highest CO readings while August had the lowest readings. 
* CO levels trend downward from October to August. They start to rise between August to October.
* CO levels are lowest on Sundays and highest on Friday. 
* CO levels are lowest between 4-5 AM and highest at 8 AM and 7 PM. This makes sense since these are rush hour times.

Let me generate a pairplot to get a better grasp of all these relationships. 

In [ ]:
aq_final = aq_filtered.iloc[:, 5:]
aq_final

In [ ]:
sns.set(font_scale=1.5)
sns.pairplot(aq_final)

Beautiful! Clearly the concentrations of pollutants are dependent on one another. To what extent though?

Strangely, the NO2 sensor shows some monotonicity with Temperature and Ambient Humidity. This is something that the other pollutants don't exhibit with those two variables. It'll be interesting to explore that. 

I've answered the time dependence questions I had regarding pollutants and time. 

Now I want to determine whether I can use this data to accurately predict pollutant concentration.

I'll be building a model for each of the following pollutants/sensors:

* PT08.S1(CO)
* PT08.S2(NMHC)
* PT08.S3(NOx)
* PT08.S4(NO2)
* PT08.S5(O3)

## Splitting Data and Building Models
I have different variables I want to predict. Furthermore, I'm interesting in testing the performance of a variety of different regressors to generate a model. In order to make things a bit cleaner code-wise, I'll write a function that will allow me to split the data for each dependent variable and test each of the 10 regressors for a model. 

In [ ]:
def model_assess(X_train, X_test, y_train, y_test, model, title = "Default"):
    
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred  = model.predict(X_test)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    results = pd.DataFrame([title,train_mse, train_r2, test_mse, test_r2]).transpose()
    results.columns = ['Method','Training MSE','Training R2','Test MSE','Test R2']
    return y_train_pred,y_test_pred, results

def multi_model_assess(df, models, y_predict):
    all_model_results = [] #This will contain all model results for each dependent variable 
    all_X_test  = []
    all_X_train = []
    all_y_test_p  = []
    all_y_train_p = []
    all_y_train = []
    #First loop will define dependent/independent variables and split data into test/training sets
    n_vars = len(y_predict)
    pbar = tqdm(range(n_vars), desc="Variable Processed", position = 0, leave = True)#Add progress bar 
    
    for dependent in y_predict:
        model_results = [] #Array with dataframes for a given dependent variable
        #Designate independent and dependent variables
        x  = df.drop([dependent], axis = 1)
        y  = df[dependent]
        #Split data into test and training sets
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
        
        #Populate the array of observed values for the dependent variable
        all_y_train.append(y_train)
        
        #Process each of the desired models
        for model, model_name in models:
            y_train_pred,y_test_pred, results = model_assess(X_train, X_test, y_train, y_test,
                                                             model, title = model_name)
            
            model_results.append(results)
            all_X_test.append(X_test)
            all_X_train.append(X_train)
            all_y_test_p.append(y_test_pred)
            all_y_train_p.append(y_train_pred)
                    
        all_model_results.append(model_results)
        pbar.update(1)
        pbar.refresh()
        
    pbar.close()   
    return all_model_results, all_X_test, all_X_train, all_y_test_p, all_y_train_p, all_y_train

Great! Now let's initiate each of the models, run them, and see how they performed!

In [ ]:
#Initiate Different Regressors for ML model
lr = LinearRegression() 
hr = HuberRegressor(epsilon=1.15, max_iter=1000)
rf = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)
gb = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gp = GaussianProcessRegressor(kernel = DotProduct() + WhiteKernel(), random_state=42)
kn = KNeighborsRegressor()
ab = AdaBoostRegressor()
sv = SVR()
dt = DecisionTreeRegressor(max_features = 'auto', max_depth=3, random_state=42)
nn = MLPRegressor(hidden_layer_sizes = 500, solver='adam', learning_rate_init = 1e-2,max_iter=500)

models =  [(lr,'Linear Regression'), 
           (hr,'Huber Regression'), 
           (rf,'Random Forest'), 
           (gb,'Gradient Boosting'),
           (gp,'Gaussian Process'), 
           (kn,'K-Neighbors'), 
           (ab,'Ada Boost'), 
           (sv,'SVR'), 
           (dt,'Decision Tree'), 
           (nn,'MLP')]

y_predict  = ['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)']

all_model_results, _, _, all_y_test_p, all_y_train_p, all_y_train = multi_model_assess(aq_final,models, y_predict)

Nice! We were able to generate a total of 10 different regressor models for each of the 5 target variables in just under 7 minutes.  Let's see how each of them performed.

## Initial Assesment of Performance for Regression Models
The initial results I have obtained here for each regressor model are based on little to no modifications from the default values for each regressor. Here I'll determine which models to refine further based using metrics like R2 and Mean Squared Error (MSE) I'll use the results of this assement to carry out a parameter grid-search/exploration to optimize the model.  

### Models for CO Concentration Prediction
The results for each of the 10 models built for the prediction of CO concentration are shown below. From 

In [ ]:
score_df_results = pd.concat(all_model_results[0], 
                        ignore_index=True).sort_values('Test R2',axis = 0, ascending = False)
score_df_results

In [ ]:
score_results_test = pd.concat(all_model_results[0], ignore_index=True)
score_results_test['Test R2'][0]

The top 3 regressor models for the prediction of CO concentration based on the R2 value of the test set resulted from:

1. Gradient Boosting
2. K-Neighbors
3. Linear Regression

I'll plot them to see how they compare

In [ ]:
#Define column with model to predict
y_predict  = ['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)']

#Model names for plot titles
models =  [(lr,'Linear Regression'), 
           (hr,'Huber Regression'), 
           (rf,'Random Forest'), 
           (gb,'Gradient Boosting'),
           (gp,'Gaussian Process'), 
           (kn,'K Neighbors'), 
           (ab,'Ada Boost'), 
           (sv,'SVR'), 
           (dt,'Decision Tree'), 
           (nn,'MLP')]

#Make labels
def make_labels(models):
    names = []
    for i in range(len(models)):
        if len(models[i][1].split()) < 2:
            names.append(models[i][1])
        else:
            names.append(''.join([s[0] for s in models[i][1].split()]))
    return names
labelList = make_labels(models)

#Specify color map to color different plots
cmap = plt.cm.get_cmap('plasma')
slicedCM = cmap(np.linspace(0, 1, len(models))) 

#Visualize results of linear regression
plt.rcParams.update({'font.size': 20})
nRows = 5 # 
nCols = 2 #

def plot_ML_model(whichVar):
    fig, axs = plt.subplots(nrows=nRows, ncols=nCols, figsize = (15,30))
    axs = axs.ravel()
    df    = pd.concat(all_model_results[whichVar], ignore_index=True)
    for k in range(10):
        color = slicedCM[k]
        yPred = all_y_train_p[k + whichVar*len(models)]
        yMeas = all_y_train[whichVar]
        label = labelList[k]
        ax    = axs[k]
        #Make scatter plot of train set and regressor model
        ax.scatter(x = yMeas, y = yPred, color=color, alpha=0.5)

        #Fit a first order polynomial (i.e. a straight line) to the regressor model 
        z = np.polyfit(yMeas, yPred, 1)
        p = np.poly1d(z)

        #Add labels and colors and stuff
        val = df['Test R2'][k] #Get the r2 value from the model results dataframe
        val = "{:.2f}".format(val)
        ax.plot(yMeas,p(yMeas),"#b20cd7", label=label+"\nr\u00b2".format(2) + " = " + str(val))
        ax.title.set_text(models[k][1]) 
        ax.set(xlabel='Train Concentration', ylabel='Predicted Concentration')
        ax.label_outer()
        ax.legend(loc="upper left")
        ax.grid(color = 'black', linestyle = '--', linewidth = 0.5)
    plt.show()

Great! Now plotting is nice and automated. Let's see the plots for the CO fits.

In [ ]:
plot_ML_model(0)

### Models for NMHC Concentration Prediction
The results for each of the 10 models built for the prediction of NMHC concentration are shown below. 

In [ ]:
score_df_results = pd.concat(all_model_results[1], 
                        ignore_index=True).sort_values('Test R2',axis = 0, ascending = False)
score_df_results

The top 3 regressor models for the prediction of CO concentration based on the R2 value of the test set resulted from:

1. Gradient Boosting
2. Huber Regression
3. Linear Regression

I'll plot them to see how they compare

In [ ]:
plot_ML_model(1)

### Models for NOx Concentration Prediction
The results for each of the 10 models built for the prediction of NOx concentration are shown below.   

In [ ]:
score_df_results = pd.concat(all_model_results[2], 
                        ignore_index=True).sort_values('Test R2',axis = 0, ascending = False)
score_df_results

The top 3 regressor models for the prediction of NOx concentration based on the R2 value of the test set resulted from:

1. Gradient Boosting
2. K-Neighbors
3. Linear Regression

I'll plot them to see how they compare

In [ ]:
plot_ML_model(2)

### Models for NO2 Concentration Prediction
The results for each of the 10 models built for the prediction of NO2 concentration are shown below.  

In [ ]:
score_df_results = pd.concat(all_model_results[3], 
                        ignore_index=True).sort_values('Test R2',axis = 0, ascending = False)
score_df_results

The top 3 regressor models for the prediction of NO2 concentration based on the R2 value of the test set resulted from:

1. Gradient Boosting
2. K-Neighbors
3. Linear Regression

I'll plot them to see how they compare

In [ ]:
plot_ML_model(3)


### Models for O3 Concentration Prediction
The results for each of the 10 models built for the prediction of NOx concentration are shown below. From  

In [ ]:
score_df_results = pd.concat(all_model_results[4], 
                        ignore_index=True).sort_values('Test R2',axis = 0, ascending = False)
score_df_results

The top 3 regressor models for the prediction of NO2 concentration based on the R2 value of the test set resulted from:

1. Gradient Boosting
2. MLP
3. K-Neighbors

The MLP did quite well for this particular dataset. I wonder why?

I'll plot them to see how they compare

In [ ]:
plot_ML_model(4)

Overall GradientBoost generated the best predictor model for each of these pollutants. Therefore, I'll devote the remainder of this project to optimizing it via a parameter grid-search 

## Parameter Space Exploration for Gradient Boosting (GB) Model
Since the GB model had the best prediction accuracy for all compounds, I will now optimize the parameters to see how much more improvement I can make on it. 

Let me start by getting the current parameters of the GB model

In [ ]:
ini_params = gb.get_params()
ini_params

Now I'll set up the grid search

In [ ]:
y_predict  = ['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)']

compound = y_predict[0]
#Designate independent and dependent variables
x  = aq_final.drop([compound], axis = 1)
y  = aq_final[compound]
#Split data into test and training sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
        
#Initiate Different Regressors for ML model
gb = GradientBoostingRegressor(random_state=42)

#Setup values for parameters
grid_values = {
               'learning_rate':[0.05,0.1, 0.15],
               'n_estimators':[150, 200, 225],
               'max_depth': [5,6,7]
              }

grid_clf_acc = GridSearchCV(gb, param_grid = grid_values, scoring = 'r2', verbose=1)
grid_clf_acc.fit(X_train, y_train)

#Predict values based on new parameters
y_train_pred = grid_clf_acc.predict(X_train)
y_test_pred  = grid_clf_acc.predict(X_test)

# New Model Evaluation metrics 
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print('r2 Score : ' + str(r2_score(y_test,y_test_pred)))

Awesome! It's finished.

## Optimized model
Let's start seeing how the model performed.

In [ ]:
#Make dataframe for grid search results
co_model = pd.DataFrame(['Gradient Boosting',train_mse, train_r2, test_mse, test_r2]).transpose()
co_model.columns = ['Method','Training MSE','Training R2','Test MSE','Test R2']
co_model

We have an R2 score of 0.91 which is not bad. Let's see which parameter combination resulted in the best model.

In [ ]:
fin_params = grid_clf_acc.best_params_
fin_params

Cool! Now I'll set the final GB model to have the values of the grid-search optimization. 

In [ ]:
optimized_GB_model = GradientBoostingRegressor(**grid_clf_acc.best_params_)

And now I'll save the optimized model into a pickle file.

In [ ]:
# save model into pickle file
with open('air_quality.pkl','wb') as f:
    pickle.dump(optimized_GB_model,f)
    
# load model from pickle file
with open('air_quality.pkl', 'rb') as f:
    gb_model = pickle.load(f)

## Using GB Model to Predict CO Concentration
Let's see how well it predicts stuff! I'll start by making predictions based on the test set. I'll see how well it performs on the entire dataset momentarily. 

In [ ]:
optimized_GB_model.fit(X_train, y_train)
co_gb_predictions = optimized_GB_model.predict(X_test)
print('Regression Model: R²={:.2f}'.format(r2_score(y_test, co_gb_predictions)))

Okay, we have R2  of 0.91. Let's compare the predictions with the observed values visualize how well the model replicates the actual concentrations of CO!

In [ ]:
final = pd.DataFrame()
final['Measured CO']   = y_test
final['Predicted CO']  = co_gb_predictions

plt.figure(figsize=(15,10))
sns.histplot(data=final)
plt.show()

Nice! The model follows the data quite well! Let's look at the distribution of the residuals. If our model is good (i.e., random error is normally distributed) then the residuals will distribute themselves in the shape of a symmetric Gaussian. 

In [ ]:
plt.figure(figsize=(15,10))
res = pd.DataFrame()
res['residuals'] = final['Measured CO'] - final['Predicted CO']
sns.histplot(data=res, kde = True)
plt.show()

Lovely! The errors in the model are normally distributed. 

Let's check for heteroskedasticity (i.e., error has non-constant variance) by plotting the residuals vs the predicted values. To avoid heteroskedasticity, the errors should not have any particular shape (they should be randomly oscillating around the mean or the zero line).

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x= final['Predicted CO'], y=res['residuals'])
plt.axhline(y=0.0, color='r', linestyle='-', linewidth=4)
plt.show()

Nice! The errors do not show heteroskedasticity. See how they oscillate around the zero line (red line). This suggests that the variances of the errors are equal.

Next let's see the normality Q-Q plot. This plot is used to determine the normal distribution of errors. If the data is perfectly normally distributed, then all the points will be contained in a straight line. Deviations from this behavior suggest non-linearity is present.

In [ ]:
fig = plt.figure(figsize = (10,4))
stats.probplot(res['residuals'], dist="norm", plot=plt)
plt.show()

Hmmm, interesting, we have some pretty apparent deviations from linearity at the upper ends of the resdiduals. However, the data is linear for most of the range. Maybe we could have been more stringent with our removal of outliers earlier?

Let me see how well it does on the entire dataset now.

In [ ]:
y_predict  = ['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)']

compound = y_predict[0]

#Designate independent and dependent variables
x  = aq_final.drop([compound], axis = 1)
y  = aq_final[compound]

#Predict concentration over entire dataset
optimized_GB_model.fit(x, y)
co_gb_pred_full = optimized_GB_model.predict(x)
print('Regression Model: R²={:.2f}'.format(r2_score(y, co_gb_pred_full)))

Very nice! R2 is 0.98 suggesting high correlation between the predicted CO concentrations from the model and the observed CO concentrations!

In [ ]:
full_final = pd.DataFrame()
full_final['Measured CO']  = y
full_final['Predicted CO'] = co_gb_pred_full

plt.figure(figsize=(15,10))
sns.histplot(data=full_final)
plt.show()

Sweet! There's not too much difference between the predicted CO concentrations and the actual ones! Let's look at the residual distribution.

In [ ]:
plt.figure(figsize=(15,10))
res_final = pd.DataFrame()
res_final['residuals'] = full_final['Measured CO'] - full_final['Predicted CO']
sns.histplot(data=res_final, kde = True)
plt.show()

Lovely! The distribution looks Gaussian.

Let's check for heteroskedasticity.

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x= full_final['Predicted CO'], y=res_final['residuals'])
plt.axhline(y=0.0, color='r', linestyle='-', linewidth=4)
plt.show()

No heteroskedasticity here. Let's look at the Q-Q plot now.

In [ ]:
fig = plt.figure(figsize = (10,4))
stats.probplot(res_final['residuals'], dist="norm", plot=plt)
plt.show()

Similar situation as when we used the test set for our predictions. Data is linear for most of the range. However, there is deviation from linearity at the both quantile ends suggesting that there is either further refinement that could be done in terms of outliers.

## Final Comparison
The plot below shows the observed vs predicted CO concentrations over the course of 1 year. The solid lines are the CO concentrations measured by the sensors while the dots are the predicted CO concentrations using the Gradient Boosting Regressor. The color scale over the x-axis is used to show the separation in months. As can be seen, the generated model and the observed values have excellent agreement with one another over the course of the entire year. 

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

plt.figure(figsize=(30,20))
maxRow = 5250
sns.scatterplot(x = aq_filtered['DateTime'][:maxRow], y = full_final['Predicted CO'][:maxRow],
             hue = aq_filtered['Month'], palette = 'twilight_shifted', legend = False,
             s=300)
sns.lineplot(x = aq_filtered['DateTime'][:maxRow], y = full_final['Measured CO'][:maxRow],
             hue = aq_filtered['Month'] , palette = 'twilight_shifted', 
             legend = 'auto')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0 ,fontsize = 30)
plt.title('Observed vs Predicted CO concentrations', fontsize = 50)
plt.ylabel('CO Concentration', fontsize = 50)
plt.xlabel('Time[yy/mm]', fontsize = 50)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.show()

## Conclusions and Final Thoughts
Models were built to predict the concentrations of 5 different pollutants (Carbon Monoxide, Nitrous Dioxide, Nitrous Oxide, Non-metal Hydrocarbons, and O3) measured over the course of 1 year using 10 different regressors. The Gradient Boosting (GB) regressor had the best performance across the board for all compounds. This model was optimized for the prediction of CO via a grid-search exploration and an R2 score of 0.98 was obtained. The GB model was used to predict the CO concentrations over the entire dataset and showed excellent agreement with the observed values. The performance of the models generated for the other pollutants is expected to be comparable to the one observed for CO.

A few things that were observed from this data are:

* October has the highest CO readings while August had the lowest readings. 
* CO levels trend downward from October to August. They start to rise between August to October.
* CO levels are lowest on Sundays and highest on Friday. 
* CO levels are lowest between 4-5 AM and highest at 8 AM and 7 PM. 
* The NO levels decrease when the other pollutant concentrations increase. NO levels are also correlated to ambient humidity. 

The final point was interesting to me. Why doesn't NO concentration increase alongside the other pollutants? 

My initial hypothesis is that NO is reacting with water and/or with some of the other polluting compounds. NO is a free radical compound (meaning that it is extremely unstable and highly reactive) formed by the reaction of nitrogen gas, N2, and oxygen gas, O2. This means that it is very likely that there is a reaction taking place that would produce something other than NO hence decreasing its concentration. A potential reaction for this would be:

1. $ 2NO(g) + O_{2}(g) -> 2NO_{2}(g) $

2. $ 2NO_{2}(g) + H_{2}O(l) -> HNO_{3}(aq) + HNO_{2}(aq)$

In the first reaction, Nitrogen Monoxide reacts with Oxygen gas to form Nitrogen Dioxide. Then,  Nitrogen Dioxide can react with water to form a mixture of Nitrous Acid and Nitric Acid. Nitric Acid is one of the components of acid rain.

This was a fun project to work on where I was able to do clean up time series data, build regressor models, and optimize them to get a accurate predictions of pollutant concentrations. 

I hope you find it interesting and/or useful!